# LTI認証連携の設定を行う -- Moodle

---

Moodleとの間で LTI 1.3による LMS(Learning Management System)との認証連携を設定する。

## はじめに

### 概要

CoursewareHub に [LTI 1.3](http://www.imsglobal.org/spec/lti/v1p3/)による LMS(Learning Management System)との認証連携を設定します。LMSを介した認証とLMSから得られた認可情報に基づく CoursewareHub へのアクセス制御を行います。このNotebookの構築手順では連携するLMSとしてMoodleを用います。

#### 基本方針

* CoursewareHubの既存の認証機能であるローカルユーザに基づく認証、学認連携に基づく認証と共存して設定可能とする
* LMSで認証されたユーザとJupyterHubユーザとの対応は LMS から得られるメールアドレスを一定ルールで変換することにより行う
    - 変換ルールは学認連携の認証と同じものを用いる
    - LMS のユーザー管理上メールアドレスとユーザーが適切に対応付くように管理されている前提とする
* CoursewareHubでの講師権限、受講者権限の制御は JupyterHub の admin 権限の有無によって制御する
    - LTI 認証連携機能自体は権限に関する制御は行わない

#### システム構成

auth-proxyコンテナのLTI認証連携機能に関する構成を下図に示します。なおローカルユーザ認証、学認連携認証については省略しています。

![lti構成](images/lti-001.png)

### 事前に準備が必要となるものについて

このNotebookを実行するにあたっての前提条件を以下に記します。

* LTI認証連携を設定するCoursewareHubとMoodleとが互いにアクセス可能であること
* CoursewareHubとLTI認証連携を行うMoodleでLTI外部ツールの設定権限を保有していること

### UnitGroup名

CoursewareHubの構築環境となるVCPのUnitGroup名を指定します。

VCノードを作成時に指定した値を確認するために `group_vars`ファイル名の一覧を表示します。

In [ ]:
!ls -1 --hide all group_vars/

上のセルの出力結果を参考にして、UnitGroup名を次のセルに指定してください。

In [ ]:
# (例)
# ugroup_name = 'CoursewareHub'

ugroup_name = 

### チェック

対象となるVCノードがAnsibleによって操作できることを確認します。

Ansibleの設定ファイルの場所を環境変数に設定しておきます。

In [ ]:
from pathlib import Path
import os

cfg_ansible = Path('ansible.cfg')
if cfg_ansible.exists():
    os.environ['ANSIBLE_CONFIG'] = str(cfg_ansible.resolve())

構築対象となる各VCノードにアクセスできることを確認します。

In [ ]:
target_hub = f'{ugroup_name}_manager'

!ansible {target_hub} -m ping

## CoursewareHubをLTI外部ツールとして追加する

CoursewareHubをLTI外部ツールとしてMoodleに登録します。

MoodleにLTI外部ツールを登録する場合、特定のコースに対して登録する方法とサイト全体に対して登録する方法があります。それぞれの設定手順を以下に示します。

### コースレベルでLTI外部ツールを追加する

コースで利用可能なLTI外部ツールを表示するには、コースのナビゲーションから[さらに]で表示されるメニューから[LTI外部ツール]を選択します。

![Moodle-サイト管理画面](images/moodle-411-01.png)

まだLTI外部ツールを登録していない場合は下図のような画面が表示されます。

![Moodle-ツール管理画面0](images/moodle-411-02.png)

[ツールを追加する]を選択するとツールの設定画面が表示されます。

![Moodle-ツール設定画面1](images/moodle-411-03.png)

「ツール設定」では上図の青枠で示した箇所への入力が必要となります。項目名を以下に示します。

* ツール名
* ツールURL
* LTIバージョン
* 公開鍵タイプ
* ログイン開始URL
* リダイレクトURI
* デフォルト起動コンテナ

各項目に入力する内容を以下に示します。

「ツール名」にはツールを識別するための名前を入力してください。

「ツールURL」には CoursewareHub のURLを入力してください。次のセルを実行すると CoursewareHubのURLが表示されます。

In [ ]:
!ansible {target_hub} -c local -a 'echo https://{{{{master_fqdn}}}}'

「LTIバージョン」は `LTI 1.3` を選択してください。

「公開鍵タイプ」は`鍵セットURL`を選択してください。

「ログイン開始URL」には CoursewareHubの OpenID Connect/Initialization Endpoint を指定します。次のセルを実行すると表示されるURLを入力してください。

In [ ]:
!ansible {target_hub} -c local -a 'echo https://{{{{master_fqdn}}}}/php/lti/login.php'

「リダイレクトURI」には CoursewareHubの Redirect Endpoint を指定します。次のセルを実行すると表示されるURLを入力してください。

In [ ]:
!ansible {target_hub} -c local -a 'echo https://{{{{master_fqdn}}}}/php/lti/service.php'

「デフォルト起動コンテナ」には「新しいウィンドウ」か「既存のウィンドウ」を選択してください。

次に「プライバシー」セクションの入力を行います。下図の青枠で示した箇所への入力が必要となります。「ランチャのメールをツールと共有する」の項目で「常に」を指定してください。

![Moodle-ツール設定画面2](images/moodle-411-04.png)

最後に「変更を保存する」を選択することでツールの登録が完了します。ツールが登録されると外部ツールの一覧画面（下図）が表示されます。

![Moodle-ツール管理画面3](images/moodle-411-05.png)

### サイト全体にツールを追加する

Moodleのサイト全体にLTI外部ツールの登録を行います。この設定を行う場合はサイトの管理者としてMoodleにログインする必要があります。

> 前節「コースレベルでLTI外部ツールを追加する」で外部ツールを登録した場合は、この節で説明する手順を実行する必要はありません。

[サイト管理]画面で[プラグイン]を選択してください。表示された画面の「活動モジュール」セクションに下図のような内容が表示されます。

![Moodle-ツール管理画面4](images/moodle-411-06.png)

「ツールを管理する」(上図赤丸部分)を選択すると、下図に示すような外部ツールの管理画面が表示されます。

![Moodle-ツール管理画面5](images/moodle-411-07.png)

「ツールを手動設定」を選択してください。「外部ツール設定」画面（下図）が表示されます。

![Moodle-ツール管理画面6](images/moodle-411-08.png)

「ツール設定」では上図の青枠で示した箇所への入力が必要となります。入力が必要となる項目名を以下に示します。

* ツール名
* ツールURL
* LTIバージョン
* 公開鍵タイプ
* ログイン開始URL
* リダイレクトURI
* デフォルト起動コンテナ

各項目に入力する内容を以下に示します（入力する値は前節「コースレベルでLTI外部ツールを追加する」で示したものと同じになります）。

「ツール名」にはツールを識別するための名前を入力してください。

「ツールURL」には CoursewareHub のURLを入力してください。次のセルを実行すると CoursewareHubのURLが表示されます。

In [ ]:
!ansible {target_hub} -c local -a 'echo https://{{{{master_fqdn}}}}'

「LTIバージョン」は `LTI 1.3` を選択してください。

「公開鍵タイプ」は`鍵セットURL`を選択してください。

「ログイン開始URL」には CoursewareHubの OpenID Connect/Initialization Endpoint を指定します。次のセルを実行すると表示されるURLを入力してください。

In [ ]:
!ansible {target_hub} -c local -a 'echo https://{{{{master_fqdn}}}}/php/lti/login.php'

「リダイレクトURI」には CoursewareHubの Redirect Endpoint を指定します。次のセルを実行すると表示されるURLを入力してください。

In [ ]:
!ansible {target_hub} -c local -a 'echo https://{{{{master_fqdn}}}}/php/lti/service.php'

「デフォルト起動コンテナ」には「新しいウィンドウ」か「既存のウィンドウ」を選択してください。

次に「プライバシー」セクションの入力を行います。下図の青枠で示した箇所への入力が必要となります。「ランチャのメールをツールと共有する」の項目で「常に」を指定してください。

![Moodle-ツール設定画面7](images/moodle-411-09.png)

最後に「変更を保存する」を選択することでツールの登録が完了します。ツールが登録されるとツールの管理画面（下図）が表示されます。表示された画面の「ツール」セクションに登録した外部ツールが表示されます。

![Moodle-ツール設定画面8](images/moodle-411-0a.png)

## CoursewareHubにLTI認証連携設定を行う

CoursewareHubに対してLTI連携のための設定を行います。

### Moodleにおける登録状況を確認する

CoursewareHubにLTI連携を設定する際に必要となる情報をMoodleの設定画面などで確認します。

LTI連携のために必要となる情報を以下に示します。

* プラットフォームID
* クライアントID
* デプロイメントID

MoodleのコースレベルでLTI外部ツールを追加した場合、Moodleのサイト全体にツールを追加した場合のそれぞれについて情報の確認方法を示します。

#### コースレベルでLTI外部ツールを追加した場合

まずLTI外部ツールの編集画面から「クライアントID」を確認します。コースのナビゲーションから[さらに]--[LTI外部ツール]を選択して LTI外部ツールの一覧画面（下図）を表示してください。

![x1](images/moodle-411-11.png)

一覧画面の「操作」からメニューを表示して「編集」を選択することで外部ツールの編集画面（下図）が表示されます。この画面に表示される「クライアントID」欄からその値を確認してください。

![x1](images/moodle-411-12.png)

また、この画面を表示した時のウェブブラウザのアドレスから「プラットフォームID」「デプロイメントID」の値を確認できます。

「デプロイメントID」の値はURLのクエリパラメータ`typeid`から確認できます。下図の例では赤丸部分から「デプロイメントID」の値が`3`であることが確認できます。また緑丸部分から「プラットフォームID」が確認できます。この値はMoodleのURLになります。

![x1](images/moodle-411-13.png)

コースレベルでLTI外部ツールを登録した場合でも、Moodleの管理者権限がある場合は次に説明する方法でも「プラットフォームID」「クライアントID」などの値を確認することができます。

#### サイト全体にツールを追加した場合

[サイト管理]--[プラグイン]--[ツールを管理する]を選択して外部ツールの管理画面（下図）を表示してください。

![x1](images/moodle-411-14.png)

上図に示した画面の「ツール」セクションに登録したツールの一覧が表示されています。登録したツールの肩に「設定詳細を表示する」リンクがあります（図中に赤丸で示している箇所）。このリンクを選択すると「ツール設定詳細」（下図）が表示されます。

![x1](images/moodle-411-15.png)

### パラメータの指定

前節までに示した手順で確認した外部ツールに関する情報を入力してください。

プラットフォームIDの値を次のセルに入力してください。

> プラットフォームIDに指定する値は、既に登録してあるものと異なる値を指定してください。同じプラットフォームIDを指定した場合、クライアントID、デプロイメントIDの値が異なる値であったとしても後から登録した値のみが有効となります。

In [ ]:
# (例)
# platform_id = 'https://moodle.example.org'

platform_id = 

クライアントIDの値を次のセルに入力してください。

In [ ]:
# (例)
# client_id = 'xxxxxxxxxxxxxxx'

client_id = 

デプロイメントIDの値を次のセルに入力してください。

In [ ]:
# (例)
# deployment_id = '1'

deployment_id = 

この節で入力したパラメータを`group_vars`ファイルに保存します。

In [ ]:
import yaml
from pathlib import Path

gvars_path = Path(f'group_vars/{ugroup_name}')
with gvars_path.open() as f:
    gvars = yaml.safe_load(f)
lti_config = gvars.get('lti_config', {})
lti_config[platform_id] = {
    'platform_id': platform_id,
    'client_id': client_id,
    'auth_login_url': f'{platform_id}/mod/lti/auth.php',
    'auth_token_url': f'{platform_id}/mod/lti/token.php',
    'key_set_url': f'{platform_id}/mod/lti/certs.php',
    'deployment_id': deployment_id,
}
gvars['lti_config'] = lti_config
with gvars_path.open(mode='w') as f:
    yaml.safe_dump(gvars, stream=f)

### CoursewareHubにLTI認証連携のための設定ファイルを配置する

前節で入力された情報をもとにLTI認証連携のための設定ファイル `lti.json` を生成し、CoursewareHub環境に配置します。

LTI関連の設定ファイルを配置、生成するための Ansible の playbook を実行します。ここで実行するplaybookでは以下の処理を行います。

* 指定されたパラメータに対応するLTIファイルを配置する
* RSAの秘密鍵ファイル`private.key`が存在していない場合は、鍵ペアの生成を行う

まず、実際に設定を変更する前にドライラン（チェックモード）でAnsibleを実行します。

In [ ]:
!ansible-playbook -CDv -l {target_hub} playbooks/setup-lti.yml || true

実際に設定ファイルの配置を行います。

In [ ]:
!ansible-playbook -Dv -l {target_hub} playbooks/setup-lti.yml

配置した設定ファイル`lti.json`の内容を確認します。

In [ ]:
!ansible {target_hub} -a 'cat {{{{jupyterhub_cfg_dir}}}}/lti/lti.json'

鍵ペアの公開鍵の内容を確認します。

> この公開鍵は将来のLMS連携機能のために使用する。認証連携には使用しない。

In [ ]:
!ansible {target_hub} -b -m shell -a \
    'openssl rsa -pubout -in {{{{jupyterhub_cfg_dir}}}}/lti/private.key 2>/dev/null'

### auth-proxyコンテナにLTI認証連携設定を反映する

前節で更新したLTIの設定ファイル`lti.json`をauth-proxyコンテナに反映します。

auth-proxyコンテナが`lti.json`を読み込み直すためにauth-proxyサービスの再起動を行います。

In [ ]:
!ansible {target_hub} -a 'docker service update --force coursewarehub_auth-proxy'

コンテナの実行状況を確認します。`auth-proxy`コンテナの状態が`Running`であり、かつ新たに起動されていることを確認してください。

In [ ]:
!ansible {target_hub} -a 'docker service ps coursewarehub_auth-proxy'

## MoodleのコースにCoursewareHubへのリンクを追加する

LTI外部ツールとして登録したCoursewareHubをアクティビティとしてコースに追加します。

### 活動チューザに表示する

サイト全体にツールを追加した場合、登録した外部ツールがコースの活動チューザに表示されていないことがあります。コースのナビゲーションから[さらに]--[LTI外部ツール]を選択してLTI外部ツールの一覧画面（下図）を表示してください。「活動チューザに表示する」のスイッチを有効となるように選択してください。

![コース画面0](images/moodle-411-20.png)

### CoursewareHubへのリンクを作成する



コースの編集モードを開始します。「編集モード」のスイッチを選択してください。

![コース画面1](images/moodle-411-21.png)

編集モードにすると表示される「活動またはリソースを追加する」を選択すると下図に示すような活動、リソースの選択画面が表示されます。選択肢の一つにLTI外部ツールを登録した時に指定した「ツール名」が表示されます。そのツール名を選択してください。

![コース画面2](images/moodle-411-22.png)

下図のような画面が表示されるので「活動名」などを入力して保存してください。

![コース画面3](images/moodle-411-23.png)

登録された外部ツールが下図のように表示されます。

![コース画面4](images/moodle-411-24.png)

### Notebookへのリンクを作成する

コースに表示する外部ツールのリンクをCoursewareHubの特定のNotebookファイルとすることができます。

特定のNotebookファイルに対するリンクとして設定するには、外部ツールの編集画面で「カスタムパラメータ」を指定します。

![カスタムパラメータ](images/moodle-411-25.png)

「カスタムパラメータ」に指定するのは以下のような内容となります。

```
notebook={リンクするNotebookのパス}
```

例えば、CoursewareHubの `ex-001.ipynb` に対するリンクを作成するには、以下のような値を指定します。

```
notebook=ex-001.ipynb
```



指定するパスはCoursewareHubの各ユーザーのホームディレクトリからの相対パスとします。 Notebookのパスを指定する場合、全てのユーザ環境でそのファイルが配置されている必要があります。Notebookが見つからない場合は404 のエラー画面が表示されます。